# Sean

In [2]:
#Import the relevant Python modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

In [3]:
#Import the relevant data
Salmon = pd.read_excel("Data/ZooPlanktonPerryData.xlsx", 5)
oldSSTs = pd.read_csv("Data/SSTs")
Salinity = pd.read_csv("Data/Entrance_Island_-_Average_Monthly_Sea_Surface_Salinities_1936-2023.csv")
Catches = pd.read_excel("Data/Copy of recreational reported fishery catch, 1953-2012.xlsx")
SeaLevel = pd.read_csv("Data/SeaLevel_Cherry_annualMean.txt", sep=";")
xl_file = pd.ExcelFile('Data/ZooPlanktonPerryData.xlsx')


FileNotFoundError: [Errno 2] No such file or directory: 'Data/SSTs'

In [ ]:
Salmon = Salmon.rename(columns={"Ocean Entry Year":"Year"})

#For each year I want a list with the SSTs for the months in ascending order. 
SSTsList = []
for year in oldSSTs["Year"].unique():
    SSTsList.append([year] + oldSSTs[oldSSTs["Year"] == year]["SST"].tolist())

SSTs = pd.DataFrame(SSTsList, columns=["Year", "SST Jan.", "SST Feb.", "SST Mar.", "SST Apr.", "SST May", "SST Jun.", "SST Jul.", "SST Aug", "SST Sep.", "SST Oct.", "SST Nov.", "SST Dec."])

#Extract the zoo plankton anomalies from the excel sheet
Original_Salmon_Sheets = {sheet_name: xl_file.parse(sheet_name) 
          for sheet_name in xl_file.sheet_names}

anom = Original_Salmon_Sheets['3. Zooplankton anomalies']
anom = anom[6:]
zoos = anom[anom.columns[3:]]
zoos["Year"] = anom["Year"].astype(int)

#Grab the relevant years from Salinity
Salinity = Salinity[55:] 
Salinity = Salinity.rename(columns={"ENTRANCE ISLAND LIGHTSTATION: AVERAGE MONTHLY SEA SURFACE SALINITIES (PSU)":"Year"})
Salinity = Salinity['Year'].astype(int)

#Next is Catches
ModernCatches = Catches[Catches["YEAR"] >= 1990]
CohoCatches = ModernCatches[ModernCatches["SPECIES_DESC"] == "COHO SALMON"]
ChinookCatches = ModernCatches[ModernCatches["SPECIES_DESC"] == "CHINOOK SALMON"]
cohoAnnualCatchList = []
chinookAnnualCatchList = []
yearList = []
for year in CohoCatches["YEAR"].unique():
    cohoAnnualCatchList.append(CohoCatches[CohoCatches["YEAR"] == year]["PIECES"].sum())
    chinookAnnualCatchList.append(ChinookCatches[ChinookCatches["YEAR"] == year]["PIECES"].sum())
    yearList.append(year)
Catches = pd.DataFrame({"Coho": cohoAnnualCatchList, "Chinook": chinookAnnualCatchList, "Year": yearList})
#Sealevel
SeaLevel = SeaLevel.drop(['Y', '000'], axis=1)
SeaLevel= SeaLevel.rename(columns={' 1985': "Year",'  6945': "Sea Level"}, errors="raise")
SeaLevel = SeaLevel[4:]


In [ ]:
#Combine the relevant data into a single data frame
newFrame1 = pd.merge(Salmon, SSTs, on="Year", how="outer")
newFrame2 = pd.merge(newFrame1, zoos, on="Year", how="outer")
newFrame3 = pd.merge(newFrame2, Salinity, on="Year", how="outer")
newFrame4 = pd.merge(newFrame3, Catches, on="Year", how="outer")
AllData = pd.merge(newFrame4, SeaLevel, on="Year", how="outer")
AllData[["Cowichan Chinook", "Harrison Chinook", "Puntledge Chinook"]] = AllData[["Cowichan Chinook", "Harrison Chinook", "Puntledge Chinook"]].fillna(0)
#Drop the years we have no viability data for
AllData = AllData.drop([28, 29, 30, 31, 32, 33])
AllDataNoNull = AllData.dropna()

## Obtained Data Frame

In [ ]:
AllDataNoNull.head(3)

# Roy

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

start_date = 1996
end_date = 2017

#(1) read sea salinity data
Salinity_file = pd.read_excel('Data/Entrance_island_salinity.xlsx') 

#(2) read Pacific decadal oscillation(PDO) data
Pdo_file = pd.read_excel('Data/pdo.xlsx','pdo')  

#(3) read Fraser river flow data
Flow_rate_file = pd.read_excel('Data/Fraser_flow_rate.xlsx','Flow rate')  

#(4) read Killer Whale data
Whale_file = pd.read_excel('Data/killerwhales_1970to2020.xlsx','killerwhales_1970to2020') 

#(5) read harbour seal population data
Seal_file = pd.read_excel('Data/harbourseals_1970to2020.xlsx','harbourseals_1970to2020')  


In [14]:
#Sea salinity is provided monthly, we will calculate yearly average. Also, nonexistent data is marked by 999.99

df_salinity = Salinity_file.replace(999.99,np.nan)
df_salinity_yr = df_salinity['YEAR']
df_salinity_val = df_salinity.drop(['YEAR'],axis=1).mean(axis = 1)
df_sea_salinity = pd.DataFrame({'Year':df_salinity_yr,'Avg Sea Salinity':df_salinity_val})
df_sea_salinity = df_sea_salinity.loc[(df_sea_salinity['Year'] >= start_date) & (df_sea_salinity['Year'] <= end_date)]
df_sea_salinity.reset_index(drop = True, inplace = True)

In [15]:
df_pdo = Pdo_file.loc[(Pdo_file['year'] >= start_date) & (Pdo_file['year'] <= end_date)]
df_pdo_clean = df_pdo.rename(columns={'year':'Year'})
df_pdo_avg = df_pdo_clean.groupby(['Year']).mean()
df_pdo_avg = df_pdo_avg.drop(columns = ['date','month'])


df_flow_rate = Flow_rate_file[(Flow_rate_file['Year'] >= start_date) & (Flow_rate_file['Year'] <= end_date) &(Flow_rate_file['PARAM'] == 1)]
df_flow_rate = df_flow_rate.drop(columns = [' ID','PARAM','MM--DD','SYM','MM--DD.1','SYM.1'])
df_flow_rate['Avg flow rate'] = 0.5*(df_flow_rate['MAX']+df_flow_rate['MIN'])
df_flow_rate.rename(columns = {'MAX':'Max flow rate','MIN':'Min flow rate'},inplace = True)
df_flow_rate.reset_index(drop = True, inplace = True)

In [16]:
df_env_factors = pd.merge(df_sea_salinity,df_pdo_avg,on = 'Year',how = 'outer')
df_env_factors = pd.merge(df_env_factors, df_flow_rate[['Year','Max flow rate','Min flow rate','Avg flow rate']], on = 'Year', how = 'outer')
print(df_env_factors)

    Year  Avg Sea Salinity       pdo  Max flow rate  Min flow rate  \
0   1996         26.720833  0.683333         8100.0          675.0   
1   1997         25.359167  1.316667        11300.0          788.0   
2   1998         27.243333 -0.478333         6710.0          600.0   
3   1999         26.195833 -1.843333        11000.0          647.0   
4   2000         27.405833 -1.125833         8000.0          470.0   
5   2001         28.385000 -1.134167         7140.0          478.0   
6   2002         27.663333 -0.439167        10600.0          491.0   
7   2003         27.967273  0.381667         7300.0          472.0   
8   2004         27.438333 -0.224167         6650.0          560.0   
9   2005         27.265000 -0.188333         7460.0          805.0   
10  2006         28.180833 -0.350833         7190.0          510.0   
11  2007         26.918333 -0.702500        10800.0          560.0   
12  2008         27.835833 -1.663333        10200.0          648.0   
13  2009         27.

In [17]:
df_whale = Whale_file.loc[(Whale_file['Year'] >= start_date) & (Whale_file['Year'] <= end_date)].drop(columns = ['SRKW','srkwSource','nrkwSource'])
df_seal = Seal_file.loc[(Seal_file['year'] >= start_date) & (Seal_file['year'] <= end_date)].drop(columns = ['nonSogNt','bcNt']).rename(columns={'year':'Year'})


df_whale.reset_index(drop = True, inplace = True)
df_seal.reset_index(drop = True, inplace = True)

df_pred_factors = pd.merge(df_seal,df_whale, on = 'Year',how = 'outer')
df_pred_factors.rename(columns = {'sogNt':'Harbour seal','NRKW':'Killer whale'},inplace = True)
df_input_variables = pd.merge(df_env_factors,df_pred_factors,on = 'Year', how = 'outer')
df_input_variables

,Year,Avg Sea Salinity,pdo,Max flow rate,Min flow rate,Avg flow rate,Harbour seal,Killer whale
0,1996,26.720833,0.683333,8100.0,675.0,4387.5,39166,214.0
1,1997,25.359167,1.316667,11300.0,788.0,6044.0,39187,218.0
2,1998,27.243333,-0.478333,6710.0,600.0,3655.0,39190,216.0
3,1999,26.195833,-1.843333,11000.0,647.0,5823.5,39190,216.0
4,2000,27.405833,-1.125833,8000.0,470.0,4235.0,39190,208.0
5,2001,28.385000,-1.134167,7140.0,478.0,3809.0,39190,200.0
6,2002,27.663333,-0.439167,10600.0,491.0,5545.5,39190,203.0
7,2003,27.967273,0.381667,7300.0,472.0,3886.0,39190,204.0
8,2004,27.438333,-0.224167,6650.0,560.0,3605.0,39190,220.0
9,2005,27.265000,-0.188333,7460.0,805.0,4132.5,39190,230.0


# Andrew

# Aycin

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")

Years

In [ ]:
#Years we concerned with 
start_year=1990
end_year=2017
year_range = range(start_year, end_year+1) 

 Target Data - Salmon Survivals

In [ ]:
#Read ZooPlanktonPerryData file
##obtain SalmonSurvivals_data
            #Take years less than or equal to 2017
                            #last row (year 2018) has only NaN


file_path = "Data/ZooPlanktonPerryData.xlsx"
selected_sheet = "5. Salmon marine survivals"  
df = pd.read_excel(file_path, sheet_name=selected_sheet)
SalmonSurvivals_data = df[df["Ocean Entry Year"]<=2017]


#Rename the columns 
abbreviation_mapping = {
    'Ocean Entry Year': 'year',
    'Cowichan Chinook': 'Cow_Ch',
    'Harrison Chinook': 'Har_Ch',
    'Puntledge Chinook': 'Pun_Ch',
    'Big Qualicum Coho': 'BQ_Coho'
}

salmonSurvivals = SalmonSurvivals_data.rename(columns=abbreviation_mapping)


# List of salmon types 
year_column = 'year'
salmonTypes_List =  [col for col in salmonSurvivals.columns if col != year_column]

Data Reads

Total Zooplankton Biomasses

In [ ]:
#Read ZooPlanktonPerryData file

##obtain  plankBiomass_data   
###take the years less than 2017 
###we drop columns that do not give biomass of zooplanktons
file_path = "Data/ZooPlanktonPerryData.xlsx"
selected_sheet = "1. Zooplankton Biomass data"  
df = pd.read_excel(file_path, sheet_name=selected_sheet)
df= df[df["yr"]<=2017]
plankBiomass_data=df.drop(columns=['key', 'survey', 'region', 'event', 'net', 'station', 'lon', 'lat', 'mon', 'day', 'time',  'twilight', 'net.type', 'diam.m', 'mesh.um', 'startz.m', 'endz.m','botz.m',
       'volfilt.m3'])

##  BiomassDefs      (if info on column names is needed)
selected_sheet = "2. Biomass data definitions"  
BiomassDefs = pd.read_excel(file_path, sheet_name=selected_sheet)



## Create a new DataFrame which contains years and corresponding total biomasses mean
#Calculate average Total Biomass for each year inside the common year range
#drop duplicates
#reset index

totalBiomass_av= pd.DataFrame()

totalBiomass_av['year']= plankBiomass_data['yr']
totalBiomass_av['Av_totalBiomass']= plankBiomass_data.groupby('yr')['Total.Biomass'].transform('mean')
totalBiomass_av.drop_duplicates(inplace=True)
totalBiomass_av.reset_index(inplace=True, drop=True)


# Create a new DataFrame for averaged values
#Group by 'yr' and calculate the annual mean of Zooplanktons
#drop duplicates
#rename the columns for convenienve
#reset_index

plankBiomassMean = pd.DataFrame()

for column in plankBiomass_data.columns:
    if column in ['yr', 'region']:
        plankBiomassMean[column] = plankBiomass_data[column]
    else:
        name = 'Av_' + column
        plankBiomassMean[name] = plankBiomass_data.groupby('yr')[column].transform('mean')


plankBiomassMean.drop_duplicates(subset=['yr'], inplace=True)
plankBiomassMean.rename(columns={'yr': 'year', 'Av_Total.Biomass': 'Av_totalBiomass'},  inplace=True)
plankBiomassMean.reset_index(drop=True, inplace=True)


### Obtained Data Frames

In [ ]:
display(plankBiomassMean.head(3))
display(totalBiomass_av.head(3))
#Note: data of totalBiomass_av appears in plankBiomassMean

Sea Level

In [ ]:
## Read the Annual Mean Sea Levels
        #there are data of four ports
#Take the years between start_year and end_year
#Take the positive values (non existent data entered as -99999
#drop the columns 2 and 3 to keep year and sea level measurement
#rename the column that contains Sea Level measurement


file_path="Data/SeaLevel_Point_Atkinson193.txt"
df=pd.read_csv(file_path, header=None,delimiter=';')
df= df[(df[0] >= start_year) & (df[0] <= end_year) & (df[1]>=0)].drop(columns=[2,3])
seaLevelPointA_clean =df.rename(columns={0: 'year', 1: 'SeaLevel_Point_Atkinson'})
#missing year 1997

file_path="Data/SeaLevel_Port_Angeles2127.txt"
df =pd.read_csv(file_path, header=None,delimiter=';')
df= df[(df[0] >= start_year) & (df[0] <= end_year) & (df[1]>=0)].drop(columns=[2,3])
seaLevelPortA_clean= df.rename(columns={0: 'year', 1: 'SeaLevel_Port_Angeles'})
#no missing year

file_path="Data/SeaLevel_Campbell_River1323.txt"
df=pd.read_csv(file_path, header=None,delimiter=';')
df= df[(df[0] >= start_year) & (df[0] <= end_year) & (df[1]>=0)].drop(columns=[2,3])
seaLevelCampbellR_clean= df.rename(columns={0: 'year', 1: 'SeaLevel_Campbell_River'})
#missing year 1995 & 1996

file_path="Data/SeaLevel_Cherry_annualMean.txt"
df= pd.read_csv(file_path, header=None,delimiter=';')
df= df[(df[0] >= start_year) & (df[0] <= end_year) & (df[1]>=0)].drop(columns=[2,3])
seaLevelCherryP_clean= df.rename(columns={0: 'year', 1: 'SeaLevel_Cherry'})
#missing year 1994


#Merge the sea Levels
#Calculate their average 
    #missing data is ignored 
        #note: if a year is missing it's only missing at most one of the ports

merged_df= pd.merge(seaLevelPortA_clean, seaLevelPointA_clean, on='year', how='outer')
merged_df = pd.merge(merged_df, seaLevelCampbellR_clean, on='year', how='outer')
seaLevels = pd.merge(merged_df, seaLevelCherryP_clean, on='year', how='outer')

seaLevels['av_SeaLevels']= seaLevels.iloc[:,1:].mean(axis=1, skipna=True)


### Obtained Data Frames

In [ ]:
seaLevels.head(3)